In [ ]:
import nltk
nltk.download('framenet_v17')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

from nltk.corpus import framenet as fn
from nltk.corpus.reader.framenet import PrettyList
from nltk import word_tokenize, sent_tokenize
from operator import itemgetter

[nltk_data] Downloading package framenet_v17 to /root/nltk_data...
[nltk_data]   Unzipping corpora/framenet_v17.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Получаем список фреймов эмоции и лексических единиц, включая глаголы

In [ ]:
emo_frames = [f.name for f in fn.frames('(?i)emotion')]
emo_verbs = [f.lexUnit for f in fn.frames('(?i)emotion')]

In [ ]:
emo_verbs

In [ ]:
emo_frames

['Cause_emotion',
 'Emotion_active',
 'Emotion_directed',
 'Emotion_heat',
 'Emotions',
 'Emotions_by_possibility',
 'Emotions_by_stimulus',
 'Emotions_of_mental_activity',
 'Emotions_success_or_failure']

Соединяем их в один словарь для удобства

In [ ]:
emo_dict = {}
for i in range(len(emo_frames)):
    emo_dict.update({emo_frames[i]: emo_verbs[i]})

In [ ]:
sents = []
for emotional_frame, verbs in emo_dict.items():
    for verb, verb_id in verbs.items(): 
        for i in range(len(verb_id.exemplars)):
            sent = verb_id.exemplars[i]
            sents.append(sent)

# pd.DataFrame(data)

In [ ]:
def get_sentence_info(sent):
    res_frames = []
    pos_dict = dict(nltk.pos_tag(word_tokenize(sent.text)))
    s = sent.annotationSet
    for n_as in range(len(s)-1):
        an_set = s[1+ n_as]
        res_frame = {}
        for i, item in enumerate(an_set.layer[:2]):
            for lab in item.label:
                try:
                    res_frame['sent_text'] = sent.text
                    element = sent.text[lab.start:lab.end+1]
                    element_tag =lab.name.lower()
                
                    if element:
                        res_frame[element_tag] = element
                except:
                    res_frame = {}
                    
                res_frames.append(res_frame)

    res = []
    for rf in res_frames:
        for k in rf:
            if k not in ('sent_text', 'target'):
                res_element = {'sent_text': rf['sent_text']}
                res_element['argument'] = rf[k]
                res_element['label'] = k
                res.append(res_element)
    
    return res

In [ ]:
import pandas as pd

In [ ]:
data = []

for sent in sents:
    
    data.extend(get_sentence_info(sent))

df = pd.DataFrame(data)

Убираем случайный мусор, который оказался в значениях 

In [ ]:
df.drop_duplicates(inplace=True)
trash = ['dep', 'ext', 'head', 'gen', 'obj']
df = df[~df['label'].isin(trash)]

In [ ]:
len(df)

6000

In [ ]:
df

,sent_text,argument,label
0,Even though Mr. Darcy is deeply offended by El...,Mr. Darcy,experiencer
1,Even though Mr. Darcy is deeply offended by El...,deeply,degree
2,Even though Mr. Darcy is deeply offended by El...,that of her family 's behaviour,event
15,Turnouts in Guardian elections were extremely ...,who,agent
16,Turnouts in Guardian elections were extremely ...,influential ratepayers,experiencer
...,...,...,...
41025,Very relevant to the changing social role of w...,by their marriages,stimulus
41035,Sweetheart was standing in the kitchen with a ...,smile on her face,expressor
41037,He raised a satisfied bright-eyed smile toward...,bright-eyed smile,expressor
41039,As Morgan walked back to the Mayfair alone he ...,he,experiencer


In [ ]:
df['argument'].value_counts()

I                                          328
he                                         143
she                                        132
He                                         126
She                                        109
                                          ... 
at being left out                            1
at being separated from their offspring      1
as he rewrote and rewrote again              1
about the way things were going              1
with stealing my money                       1
Name: argument, Length: 3517, dtype: int64

In [ ]:
df.to_csv('framenet_dataset.csv', encoding='utf-8', index=False)